# Project N°1 
Make by:

Jose Luis Hincapie Bucheli - 2125340

Carlos Andrés Hernández - 2125653

## Require Libraries

In [1]:
%%writefile requirements.txt
numpy==1.25.2
prettytable

Overwriting requirements.txt


In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Discretization of the initial problem.

In [3]:
import numpy as np
#from auxiliares import limpiar_pantalla
from prettytable import PrettyTable 

# Se define la malla a tratar con las condiciones iniciales

v_0 = 1 # Velocidad inicial en x

mallax = np.array(
    [[1,  0,   0,   0,   0,   0,   0,  0],
    [1, "w1", "w2", 0, 0, "w3", "w4", 0],
    [1, "w5", "w6", 0, 0, "w7", "w8", 0],
    [1, "w9", "w10", "w11", "w12", "w13", "w14", 0],
    [1, "w15", "w16", 0, 0, "w17", "w18", 0],
    [1, "w19", "w20", 0, 0, "w21", "w22", 0],
    [1,  0,   0,   0,   0,   0,   0,  0]])

mallay = np.array(
    [[0,  0,   0,   0,   0,   0,   0,  0],
    [0, "w1", "w2", 0, 0, "w3", "w4", 0],
    [0, "w5", "w6", 0, 0, "w7", "w8", 0],
    [0, "w9", "w10", "w11", "w12", "w13", "w14", 0],
    [0, "w15", "w16", 0, 0, "w17", "w18", 0],
    [0, "w19", "w20", 0, 0, "w21", "w22", 0],
    [0,  0,   0,   0,   0,   0,   0,  0]])


r_matrix = np.zeros((22, 23), dtype=int) # Almacenar los resultados finales de la discretización

# Funtion that reshape results in a finish matrix the last colum are the numeric values

def fixMatrix(values, count):
    global r_matrix
    acum = 0
    for value in values:
        if value[0] == 'w':
            match np.where(values == value)[0][0]:
                case 0:
                    r_matrix[count][int(value[1:])-1] = 1
                case 1:
                    r_matrix[count][int(value[1:])-1] = -8
                case 2:
                    r_matrix[count][int(value[1:])-1] = 3
                case 3:
                    r_matrix[count][int(value[1:])-1] = 3
                case 4:
                    r_matrix[count][int(value[1:])-1] = 1
        else:
            match np.where(values == value)[0][0]:
                case 0:
                    acum += int(value)*1
                case 1:
                    acum += int(value)*-8
                case 2:
                    acum += int(value)*3
                case 3:
                    acum += int(value)*3
                case 4:
                    acum += int(value)*1
    r_matrix[count][-1] = acum


# Funtion that find the positions needed in navier-stokes using Finite differences
def discretization(malla):
    count = 0
    for row in range(7):
        for colum in range(8):
            if malla[row][colum][0] == 'w':
                temp = np.array([malla[row][colum+1], malla[row][colum],
                                malla[row][colum-1], malla[row-1][colum], malla[row+1][colum]])
                fixMatrix(temp, count)
                count += 1


if __name__ == "__main__":
    discretization(mallax)

    column_names = [f"W{i}" for i in range(1, 24)]
    column_names[-1] = "T.I"
    
    # Create a table to format de r_matrix table
    table = PrettyTable()
    table.field_names = [""] + column_names 
    
    for i, row in enumerate(r_matrix, start=1):
        equation_label = f"Ecuación {i}"
        row_as_strings = [str(value) for value in row]
        table.add_row([equation_label] + row_as_strings)
        

    print(table)

+-------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|             | W1 | W2 | W3 | W4 | W5 | W6 | W7 | W8 | W9 | W10 | W11 | W12 | W13 | W14 | W15 | W16 | W17 | W18 | W19 | W20 | W21 | W22 | T.I |
+-------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|  Ecuación 1 | -8 | 1  | 0  | 0  | 1  | 0  | 0  | 0  | 0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  3  |
|  Ecuación 2 | 3  | -8 | 0  | 0  | 0  | 1  | 0  | 0  | 0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |
|  Ecuación 3 | 0  | 0  | -8 | 1  | 0  | 0  | 1  | 0  | 0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |
|  Ecuación 4 | 0  | 0  | 3  | -8 | 0  | 0  | 0  | 1  | 0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0 

# Second Part Of The Solution

To solve the problem of finding the solution for the parameter variables, use Successive Over-Relaxation (SOR) to aproch the solution faster.

- First check if the matrix is strictly diagonally dominant

In [4]:
'''
Funtion that checks if a matrix is diagonally dominant
X = Matrix
'''
def dd(X):
    M = np.delete(X,-1,1)
    D = np.diag(np.abs(M)) # Find diagonal coefficients
    S = np.sum(np.abs(M), axis=1) - D # Find row sum without diagonal
    if np.all(D >= S):
        print( 'matrix is diagonally dominant')
    else:
        print ('NOT diagonally dominant')

dd(r_matrix)

matrix is diagonally dominant


Now that the conditions are meet and we know the method will converge in a solution we use SOR


In [5]:
'''
Funtion that use jacobi method in a especific row
R = Row to apply jacobi
S = array of initial solutions for the row
nn = interation value, in other words for which variable we want to apply jacobi.
'''
def jacobi(R,S,nn):
    acum =0
    c = -R[-1]/R[nn]
   # print(f"{-R[-1]}/{R[nn]} = {c}")
    acum += c
    for i in range(len(R)-1):
        #print(f"{-R[i]} / {R[nn]}*{S[i]} = {(-R[i]/ R[nn]) * S[i]}")
        acum+= ( (-R[i]/ R[nn]) * S[i] ) if i != nn else 0
    return acum

In [10]:
jacobi(r_matrix[-1],np.zeros(22),21)

0.0

In [52]:
'''
Funtion that checks if the given solutions satisfy the tolerance
S = Preview solution
NS = New Solution
tol = tolerance
'''
def check_tolerance(S,NS,tol):
    #Infinity Norm
    error = ( np.max(np.abs(NS)) - np.max(np.abs(S)) )/ np.max(np.abs(NS))
    print("Error: ",error)
    if error > tol : return True 
    else: return False 
    

In [57]:
'''
Funtion that use Successive Over-Relaxation in an especific matrix.
M = Matrix of coeficients with idependent vector  
omega = stride for faster convergence.
tol = tolerance
'''

def SOR(M,omega,tol):
    l = len(M)
    S= np.zeros(l)
    OS = np.copy(S)
    acum = 0
    for i in range(l):
        c = -M[i][-1]/M[i][i]
        S[i] = c    
    while check_tolerance(OS,S,tol):
        acum += 1
        OS = np.copy(S)
        for i in range(l):
            S[i] = (1 - omega) * S[i] + omega * jacobi(M[i],S,i)
    print(f"Number of iterations: {acum}")
    print("Solutions: ")
    print(S)

In [56]:
SOR(r_matrix,1.2,0.1)

Error:  1.0
Error:  0.5012888566111955
Error:  0.12072251780308257
Error:  0.004822101940646059
Number of iterations: 3
Solutions: 
[0.49391285 0.23860971 0.00099771 0.0013469  0.71613287 0.43120175
 0.00612231 0.00512097 0.82449689 0.57724476 0.22739601 0.09039226
 0.04100276 0.01911735 0.85932119 0.59997487 0.01832543 0.01500941
 0.75815391 0.50843105 0.00782167 0.00863033]
